This notebook accompanies the web tutorial. It trains Multilayer Perceptrons with
different depths (1, 2, 4 hidden layers) on the Fashion-MNIST dataset and
reproduces all plots shown in the webpage.

GitHub repository: YOUR_GITHUB_REPO_URL_HERE

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)

# Load dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalise
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Flatten
x_train = x_train.reshape(-1, 28 * 28)
x_test  = x_test.reshape(-1, 28 * 28)

# Train/val split
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train_small = x_train[:-10000]
y_train_small = y_train[:-10000]

x_train_small.shape, x_val.shape, x_test.shape


ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
def create_mlp(num_hidden_layers=1, hidden_units=128, input_dim=784, num_classes=10):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))

    for _ in range(num_hidden_layers):
        model.add(layers.Dense(hidden_units, activation="relu"))

    model.add(layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


In [ ]:
depths = [1, 2, 4]
histories = {}
test_scores = {}

for d in depths:
    print(f"\nTraining MLP with depth = {d}")
    model = create_mlp(num_hidden_layers=d)

    history = model.fit(
        x_train_small, y_train_small,
        validation_data=(x_val, y_val),
        epochs=20,
        batch_size=128,
        verbose=2,
    )

    histories[d] = history.history

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    test_scores[d] = (test_loss, test_acc)
    print(f"Depth {d}: test accuracy = {test_acc:.4f}")


In [ ]:
plt.figure(figsize=(7, 5))

colors = {1: "tab:blue", 2: "tab:orange", 4: "tab:purple"}

for d in depths:
    acc = histories[d]["accuracy"]
    val_acc = histories[d]["val_accuracy"]
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, val_acc, linestyle="--", color=colors[d], label=f"val (depth={d})")
    plt.plot(epochs, acc, linestyle="-",  color=colors[d], alpha=0.6, label=f"train (depth={d})")

plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy for Different MLP Depths")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("figures/acc_curves.png", dpi=200)
plt.show()


In [ ]:
plt.figure(figsize=(5, 4))

depth_list = list(test_scores.keys())
test_accs = [test_scores[d][1] for d in depth_list]

bars = plt.bar([str(d) for d in depth_list], test_accs, color=[colors[d] for d in depth_list])
plt.xlabel("Number of hidden layers")
plt.ylabel("Test accuracy")
plt.ylim(0.7, 1.0)
plt.title("Test Accuracy vs MLP Depth on Fashion-MNIST")

for bar, acc in zip(bars, test_accs):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.005,
             f"{acc:.3f}", ha="center", va="bottom", fontsize=8)

plt.tight_layout()
plt.savefig("figures/test_acc_bars.png", dpi=200)
plt.show()
